In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
#이벤트 시간 처리
spark.conf.set("spark.sql.shuffle.partitions", 5)
static = spark.read.json("file:///home/ubuntu/Spark-The-Definitive-Guide/data/activity-data")
streaming = spark\
  .readStream\
  .schema(static.schema)\
  .option("maxFilesPerTrigger", 10)\
  .json("file:///home/ubuntu/Spark-The-Definitive-Guide/data/activity-data")

In [3]:
#이벤트 시간 윈도우
withEventTime = streaming.selectExpr(
  "*",
  "cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")

In [4]:
#텀블링 윈도우
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes")).count()\
  .writeStream\
  .queryName("pyevents_per_window")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [6]:
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes"), col("User")).count()\
  .writeStream\
  .queryName("pyevents_per_window_1")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [7]:
#슬라이딩 윈도우
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
  .count()\
  .writeStream\
  .queryName("pyevents_per_window_2")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [8]:
#워터마크로 지연 데이터 제어하기
from pyspark.sql.functions import window, col
withEventTime\
  .withWatermark("event_time", "30 minutes")\
  .groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
  .count()\
  .writeStream\
  .queryName("pyevents_per_window_3")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [9]:
#스트림에서 중복 데이터 제거하기
from pyspark.sql.functions import expr

withEventTime\
  .withWatermark("event_time", "5 seconds")\
  .dropDuplicates(["User", "event_time"])\
  .groupBy("User")\
  .count()\
  .writeStream\
  .queryName("pydeduplicated")\
  .format("memory")\
  .outputMode("complete")\
  .start()